<img src="./images/instagram_header.png" align="left" style="margin-bottom: 20px"/>

<h2> Web Appendix - Data Collection & Preparation </h2>

<p style="clear: both;">This online appendix complements the master thesis "Goodbye Likes, Hello Mental Health: How Hiding Like Counts Affects User Behavior & Self-Esteem":</p> 

<p><i>Likes are widely available on social network services and are known to influence people’s self-image. An emerging literature has started to look at potential detrimental effects of social media use among teenagers. We study how Instagram users’ posting frequency, variety, like behavior, and relative self-esteem are affected by an intervention in which like counts were hidden in selected treatment countries. Using a unique panel data set of individual users’ Instagram posts across multiple years, we find evidence that users posted more frequently and more varied than in the months prior to the intervention. On the other hand, the number of likes decreases as people are no longer influenced by others’ evaluations, especially among users with a small following. Further, in an experiment we show that the number of likes people see on others’ posts affects their relative self-esteem, and that users are more likely to self-disclose once they rate themselves more positively. These results are critical to understanding the dynamics on visual-based social media in order to foster a healthy online environment.</i></p>

<p>In this notebook, we perform the following steps (run this .ipynb-file locally for clickable anchors): </p>

A. [Instagram Influencer Seed](#instagram-influencer-seed)  
B. [Instagram Consumers Selection](#instagram-consumers-selection)  
C. [Collect & Preprocess Instagram Data](#preprocess-instagram-data)  
D. [Computer Vision](#computer-vision)  
E. [Cosine Similarity & Image Similarity](#cosine-similarity)  
F. [Outlier Screening](#outlier-screening)  
G. [Propensity Score Matching](#propensity-score-matching)  

<i> Note: Steps H and I (data analysis) can be found over <a href="https://github.com/RoyKlaasseBos/Hiding-Instagram-Likes/blob/master/Web_Appendix_Data_Analysis.ipynb">here</a>. </i>

In [1]:
import os, psycopg2, pickle, pandas as pd, requests, json
from sklearn.metrics.pairwise import cosine_similarity 
from sqlalchemy import create_engine

# define paths to PostgresSQL database (instructions on how to setup the environment variables can be found in the README file)
host = os.environ['INSTAGRAM_DB_URL'] 
password = os.environ['INSTAGRAM_DB_KEY']
connection = psycopg2.connect(host=host, user='postgres', password=password, dbname='postgres');
cursor = connection.cursor()
connection_string = 'postgresql+psycopg2://postgres:' + password + '@' + host + '/postgres'
engine = create_engine(connection_string)

# support R in Jupyter Notebook
%load_ext rpy2.ipython

In [2]:
%%R
# in case you are unable to select a CRAN mirror in Jupyter Notebook: open RStudio and install the packages there which resolves the issue.
install.packages(c("RPostgreSQL", "Matching"))
library(RPostgreSQL)
library(Matching)

host = Sys.getenv(c("INSTAGRAM_DB_URL"))
password = Sys.getenv(c("INSTAGRAM_DB_KEY"))

drv = dbDriver("PostgreSQL")
con = dbConnect(drv, host=host, 
                port='5432', dbname='postgres',
                user='postgres', password=password)

R[write to console]: Loading required package: DBI

R[write to console]: Loading required package: MASS

R[write to console]: ## 
##  Matching (Version 4.9-6, Build Date: 2019-04-07)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##




<a id="instagram-influencer-seed"></a>
### A. Instagram Influencer Seed

<a href="https://hypeauditor.com/top-instagram/">HypeAuditor</a> lists the top influencers by  category by country. The ranking is updated daily and takes into account quality audience and authentic engagement to control for bots and inactive accounts. For each listing the number of followers from a given country and the total number of followers are indicated. First, we scraped all listings for each category and country combination in February 2020. The number of listings for each combination varies depending on the volume of influencers in the domain. In 45 cases (0.3%) the ``followers_from_country``  column was missing because the data was unavailable on HypeAuditor. These records have been excluded from our analysis. Second, we divide both metrics by one another to derive the *purity*. That is, the percentage of influencers' followers from a given country. Third, we exclude followers whose purity is below 50%. As a result, we end up with a list of influencers (N = 5391) whose main following base is located in a single country (41% of all top influencers). Fourth, we sort the influencers by purity and pick the top 20 influencers for each country in our dataset.

<img src="./images/hypeauditor.png" align="left" />

In [3]:
def load_data(table):
    '''pull in all data from a table'''
    cursor.execute("SELECT * FROM " + table )
    data = pd.DataFrame(cursor.fetchall())
    data.columns = [desc[0] for desc in cursor.description]
    return data

def pickle_files(pickle_name, df):
    '''store output of data frame as pickle'''
    with open(pickle_name, 'wb') as f:
        pickle.dump(df, f)
        
def convert_follower_counts(df, column):
    '''convert string follower counts (e.g., 3K) into numeric (e.g., 3000)'''
    for counter in range(len(df)): 
        if "M" in df.loc[counter, column]:
            df.loc[counter, column] = float(df.loc[counter, column].replace("M", "")) * 1000000        
        elif "K" in df.loc[counter, column]:
            df.loc[counter, column] = float(df.loc[counter, column].replace("K", "")) * 1000
    return df

def extract_country(df): 
    '''extract country from URL (e.g., https://hypeauditor.com/top-instagram-beer-wine-spirits-brazil/ --> brazil)'''
    countries = ["australia", "brazil", "canada", "china", "france", "germany", "hong-kong", "india", "indonesia", \
                 "italy", "malaysia", "mexico", "russia", "saudi-arabia", "slovakia", "spain", "switzerland", "ukraine", \
                 "united-arab-emirates", "united-kingdom", "united-states"]
    for counter in range(len(df)): 
        for country in countries: 
            if country in df.loc[counter, 'url']:
                df.loc[counter, 'country'] = country
    return df


def calculate_purity(df):
    '''determine the percentage of influencers' followers from a given country'''
    for counter in range(len(df)):
        df.loc[counter, 'percentage_country'] = float(df.loc[counter, 'followers_from_country']) / float(df.loc[counter,'total_follower'])
    return df


# import data
df = load_data("hypeauditor")

# add country column to data
df = extract_country(df)

# convert follower counts to numeric
df = convert_follower_counts(df, 'followers_from_country')
df = convert_follower_counts(df, 'total_follower')

# exclude records for which either the followers from country or total follower count is missing
df = df[(df['followers_from_country'] != "") & (df['total_follower'] != "")].reset_index(drop=True)

# add purity measure
df = calculate_purity(df)

# exclude influencers whose purity is below 50%; mean purity increases from 37.1% to 70.1%
df = df.loc[df.percentage_country > .5]

# select top influencers from each country whose purity is highest (mean purity: 82.8%)
top_20 = df.groupby('country')['percentage_country'].nlargest(20)

# obtain the usernames belonging to these influencers
indices = [top_20.index[counter][1] for counter in range(len(top_20.index))] 
selected_df = df.loc[indices, ['username', 'country', 'percentage_country']]
# selected_df.to_sql('influencer_country_purity', engine, index=None, if_exists='replace')

<a id="instagram-consumers-selection"> </a>
### B. Instagram Consumers Selection

For each selected influencer we collect a list of their followers using [Phantombuster](https://phantombuster.com/automations/instagram/7085/instagram-profile-scraper) (figure below), draw a random sample of  followers, and validate their country of origin in order to construct our dataset of consumers. Our sample includes personal accounts owned by individual users who do not engage in commercial activities on Instagram and whose country of origin is validated. Furthermore, we validate the user's country of origin. The step-by-step user screening procedure with examples can be found [here](https://github.com/RoyKlaasseBos/Hiding-Instagram-Likes/blob/master/User_Screening_Manual.pdf). Finally, we end up with a list of 40 accounts by country:</p>


| Country | Type | #Accounts |
| ------ | ------ |------ |
| Australia | Treatment | 40 |
| Canada | Treatment | 40 |
| France | Control | 40 |
| Germany | Control | 40 |
| Italy | Treatment | 40 |
| Netherlands | Control | 40 |
| Spain | Control | 40 |
| United Kingdom | Control | 40 |


<a id="preprocess-instagram-data"></a>
### C. Collect & Preprocess Instagram Data
In step 1 and 2 we created a list of Instagram usernames of which we collected historical post data using [Instagram Scraper](https://github.com/arc298/instagram-scraper). This is a command-line application written in Python to obtain user information, social relationship information, and photo information. Since the photo and video files attached to each post take up a significant amount of memory, we only store the links to the online media files. More specifically, we run the command below to collect the post information of usernames in `FILE_NAME.txt` (i.e., text file that contains all usernames in our sample). 

<img src="./images/instagram_scraper.png" alt="Instagram Scraper Github" align="left" width="600px">

`instagram-scraper -f FILE_NAME.txt --media-types none --media-metadata --profile-metadata -T {username}_{urlname}`


The scraping process yields a separate JSON-file for each account which requires further preprocessing for follow-up analysis. For each user we extracted post and user level data and stored it into a dataframe which we then pushed to a local database. 

In [4]:
def parse_posts(df):
    '''parse the json file for each username in the text file and store preprocessed records in a data frame'''
    
    # declare dataframes for post data (posts) and arrays for user accounts which are private (i.e., user data could not be scraped)
    posts = pd.DataFrame()
    private = []

    # test if user profile is publicly available
    for user in range(len(df)):
        d = df.loc[user, 'json']
          
        try:        
            # post level data
            shortcode = [d[counter]['shortcode'] for counter in range(len(d))]
            description = [d[counter]['edge_media_to_caption']['edges'][0]['node']['text'] if len(d[counter]['edge_media_to_caption']['edges']) > 0 else "NA" for counter in range(len(d))]
            total_likes = [d[counter]['edge_media_preview_like']['count'] for counter in range(len(d))]
            total_comments = [d[counter]['edge_media_to_comment']['count'] for counter in range(len(d))]
            hashtags = [d[counter]['tags'] if type(d[counter].get('tags')) == list else "NA" for counter in range(len(d))]
            content_type = [d[counter]['__typename'] for counter in range(len(d))]
            timestamp = [d[counter]['taken_at_timestamp'] for counter in range(len(d))]
            video_views = [d[counter]['video_view_count'] if type(d[counter].get('video_view_count')) == int else 0 for counter in range(len(d))]
            media1 = [d[counter]['urls'][0] for counter in range(len(d))]

            posts_temp = pd.DataFrame({
                           "username": d[0]['username'],
                           "shortcode": shortcode, 
                           "description": description, 
                           "total_likes": total_likes,
                           "total_comments": total_comments,
                           "hashtags": hashtags,
                           "content_type": content_type,
                           "timestamp": timestamp, 
                           "video_views": video_views,
                           "media1": media1 # for image caroussels we focused on the first photo/video
                          })

            posts = pd.concat([posts_temp, posts]).reset_index(drop=True)
            
        except: 
            private.append(user)

    # convert epoch time to regular timestamp
    posts['timestamp'] = pd.to_datetime(posts['timestamp'], unit='s')

    # add regular date (without time)
    posts['date'] = posts['timestamp'].dt.date

    return posts, private


def parse_profiles(df):
    '''parse the personal biography for each username in the text file and store preprocessed records in a data frame'''
    profile = pd.DataFrame()

    # test if user profile is publicly available
    for user in range(len(df)):
        d = df.loc[user, 'json']

        # user level profile data
        try: 
            followers_count = d['info']['followers_count']
            following_count = d['info']['following_count']
            posts_count = d['info']['posts_count']
            biography = d['info']['biography']
            full_name = d['info']['full_name']

            profile_temp = pd.DataFrame({
                "username": d['username'][0],
                "followers_count": followers_count,  
                "following_count": following_count, 
                "posts_count": posts_count,
                "biography": biography, 
                "full_name": full_name 
            })
            
        except: 
            pass

        profile = pd.concat([profile_temp, profile]).reset_index(drop=True)

    return profile

# for all usernames process JSON files and push normalized data to SQL database (takes a while to run!)
temp_output = parse_posts(load_data('consumers_posts_json'))
temp_posts = temp_output[0]
temp_private = temp_output[1]
temp_profile = parse_profiles(load_data('consumers_profile_json'))
        
#temp_posts.to_sql('consumers_posts', engine, index=None, if_exists='replace')
#temp_profile.to_sql('consumers_profile', engine, index=None, if_exists='replace')

<a id="computer-vision"></a>
### D. Computer Vision
We use Azure Cognitive Services Computer Vision Application Programming Interface ([API](https://azure.microsoft.com/en-us/services/cognitive-services/computer-vision/)) to analyze image content. For every image, the API returns a vector of tags and confidence scores (figure below). First, we make an API request and pickle all output data for further analysis. Second, we compute image similarity within and between-subjects using image tags data. Note that Instagram image URLs are valid for a limited amount of time. The code sample below, therefore, only runs for recently scraped data. Expired URLs are printed in the console.

<img src="./images/vision_api_example.png" align="left" alt="Computer Vision API Example (tags)">

In [5]:
if 'COMPUTER_VISION_SUBSCRIPTION_KEY' in os.environ:
    subscription_key = os.environ['COMPUTER_VISION_SUBSCRIPTION_KEY']
else:
    print("\nSet the COMPUTER_VISION_SUBSCRIPTION_KEY environment variable.\n**Restart your shell or IDE for changes to take effect.**")
    sys.exit()

if 'COMPUTER_VISION_ENDPOINT' in os.environ:
    endpoint = os.environ['COMPUTER_VISION_ENDPOINT']
    
analyze_url = endpoint + "vision/v2.1/analyze"


def process_image(temp_df, categories=True):
    '''obtain categories or tags image data from all images in dataframe using Azure Cognitive Services'''
    df = pd.DataFrame(columns=['uri', 'timestamp', 'category', 'score'])
    
    for counter in range(len(temp_df)):
        image_url = temp_df.loc[counter, 'media1']
        time_stamp = temp_df.loc[counter, 'timestamp']
    
        headers = {'Ocp-Apim-Subscription-Key': subscription_key}
        data = {'url': image_url}
        params = {'visualFeatures': 'Categories'} if categories else {'visualFeatures': 'Tags'}
        
        try: 
            response = requests.post(analyze_url, headers=headers,
                                 params=params, json=data)
            output = response.json()

            if categories: 
                for category in output['categories']: 
                    df = df.append(
                        dict(
                            uri = image_url,
                            timestamp = time_stamp,
                            category = category['name'],
                            score = category['score'],
                        ), ignore_index = True)

            else: 
                for tag in output['tags']: 
                    df = df.append(
                            dict(
                                uri = image_url,
                                timestamp = time_stamp,
                                category = tag['name'],
                                score = tag['confidence'],
                            ), ignore_index=True)           
                    
        except: 
            #image url expired
            print(image_url)             
            
    return df

# to save time and computing resources we only collect image tags data among users selected after matching
# in step 7 we describe the propensity score matching procedure
consumers_psm_query = load_data("consumers_psm")
consumers_selected = consumers_psm_query['username']

# consumers' post level data
temp_posts = load_data("consumers_posts")

# The image urls in our sample are expired which implies we cannot access nor analyze these images anymore 

# for consumer in consumers_selected:
#     if not os.path.isfile('./pickles/image_output/Azure_Tags/' + consumer + '.pickle'):
#         temp = temp_posts.loc[(temp_posts.username == consumer) & (temp_posts.content_type != "GraphVideo")].reset_index(drop=True) 
#         image_tags = process_image(temp, False)
#         image_tags.to_sql('image_tags', con=engine, index=None, if_exists='append') 
#         pickle_files('pickles/image_output/Azure_Tags/' + consumer + '.pickle', image_tags)

<a id='cosine-similarity'> </a>
### E. Cosine Similarity & Image Similarity

#### E.1 Cosine Similarity
To illustrate how the cosine similarity scores are derived, we go over a fictitious example for the within-subject design. Let's assume a user posted two pictures of which we want to compute the cosine similarity: 

<img src="./images/cosine_similarity.jpg" align="left" alt="Cosine Similarity Example"/>

As follows from the figure the computer vision algorithm API returned three tags for both pictures. The first picture contains a group of people watching the sunset together, and the second picture also shows a group of people standing in a forest. To account for uncertainty each of these tags is associated with a confidence score, which we can write down in matrix notation as follows: 

In [6]:
pictures = pd.DataFrame([[0.67, 0.93, 0.89, 0.00], [0.74, 0.92, 0.00, 0.88]], columns=['people_group', 'outdoor', 'sunset', 'forest'], index=['picture1.jpg', 'picture2.jpg'])
pictures

,people_group,outdoor,sunset,forest
picture1.jpg,0.67,0.93,0.89,0.00
picture2.jpg,0.74,0.92,0.00,0.88


In the first and second row `forest` and `sunset` were assigned a confidence score of `0.00` respectively as these tags were not present in the images. Next, we perform the cosine similarity operation which measures the angle between two vectors and determines whether two vectors are pointing in the same direction. More specifically, we multiply the confidence scores of pictures 1 and 2 for each image tag (e.g., for people: 0.67 x 0.74) and divide by the multiplication of the length of both vectors. Mathematically, this can be denoted as: 
$$sim(r,c)=  (r \cdot c)/(\left\Vert r \right\Vert \cdot \left\Vert c \right\Vert)$$ 
<br /> 
Here $r$ and $c$ are the image vectors for picture 1 and 2 respectively, and $||r||$ is defined as $\sqrt{r_1^2+r_2^2+ ... + r_n^2}$. A larger confidence score has a larger weight and more overlapping image tags gives a higher cosine similarity score. Filling in the confidence scores above, we find a cosine similarity of `0.63` between picture 1 and 2. The diagonal contains 1s as comparing any image with itself always yields a cosine similarity of 1. 

In [7]:
cosine_similarity(pictures)

array([[1.        , 0.63240507],
       [0.63240507, 1.        ]])

Next, we explain how we can apply cosine similarity transformations to address whether the variety of posts changes after the introduction of the intervention. First, we compute the cosine similarity between pictures taken before [after] the intervention with all other pictures taken before [after] the intervention. This gives a mean image similarity score by user: 
<br/>

| username | before_after | image_similarity |
| -------  | -------- | --------- | 
| alexanderkuckart | before | 0.2140 | 
| alexanderkuckart | after | 0.1984 | 
| ... | ... | ... | 
| xannabellex27 | before | 0.3134 | 
| xannabellex27 | after | 0.3087 | 

Again, let's consider a hypothethical user who used to only share like-seeking selfies on Instagram. After hiding like counts about half of the posts still include selfies, but the remaining posts include other subjects (e.g. scenery). This implies that the image similarity would drop since the cosine similarity of a blend of selfies and scenery photos is lower than the cosine similarity among a homogeneous sample of selfies. 

#### E.2 Image Similarity

First, we make within-subject comparisons to address whether the variety of posts changes after the introduction of the intervention. Second, we make between-subjects comparisons to determine whether treated users share more unique content relative to others. 

*Within-subject similarity*  
For each user i we compute the cosine similarity between pictures taken by the same user $i$. We distinguish between pictures taken before ($1_{before}$…$n_{before}$) and after  
($1_{after}$…$n_{after}$) the intervention. This yields a similarity matrix in which each picture *before* [*after*] the intervention is compared with all pictures *before* [*after*] hiding like counts (i.e., white squares in figure below). Each row ($r$) and column ($c$) name present a picture from user $i$ in $k$, where $k$ can take on the value *before* or *after*. Given these two separate subsets $k$, we calculate how similar each picture on average is to all other pictures in the same subset. That is, for each row we take the row average excluding the diagonal values. Finally, we aggregate the results across all rows in $k$:


$\omega_{ik} = \frac{1}{n_{ik}(n_{ik}-1)}\sum_{r=1}^{n} \sum_{c=1|c \neq r}^{i-1} sim(r_{ik},c_{ik})$ 

The row and column names represent pictures before and after the intervention for user *i* ($\mu_i$). Values in the matrix denote the cosine similarity for each picture pair (only the diagonal of 1s have been reported).  For the purpose of this analysis we restrict ourselves to the white squares in the top left and bottom right quadrant of the figure. Within these areas we compute row means (excluding the diagonal values) to determine how similar a given picture is to all other pictures in the same subset on average. Thereafter, we derive the before and after within-subject similarity by taking the average of the row means in the top left and bottom right squares, respectively. Note: calculating column means, rather than row means, yields identical outcomes. 

<img src="./images/within_subjects_similarity.png" align="left"/>


In [8]:
def within_subject_similarity(consumers, consumers_selected):
    '''compute the within-subject similarity from image tags before and after the intervention'''

    for consumer in consumers_selected:
        w_similarity_scores = pd.DataFrame() 
        
        image_data = load_data("image_tags WHERE username = '" + consumer + "'")
            
        consumers_df = pd.merge(image_data, consumers, left_on='uri', right_on='media1')[['uri', 'category', 'score', 'before_after']]

        # turn image categories/tags into a matrix (rows: images, columns: categories/tags) and order by date (year & month)
        tags_matrix = consumers_df.pivot_table(index=["before_after", "uri"], columns="category")
        tags_matrix = tags_matrix.fillna(0)
        similarity = cosine_similarity(tags_matrix)

        before_intervention = len(tags_matrix.loc[('before')])
        after_intervention = len(tags_matrix.loc[('after')])

        before_similarity = pd.Series([similarity[counter][list(range(0, counter)) + list(range(counter + 1, before_intervention))].mean() for counter in range(before_intervention)]).mean()
        w_similarity_scores.loc[len(w_similarity_scores) + 1, ['username', 'before_after', 'image_similarity']] = [consumer, 'before', before_similarity]

        after_similarity = pd.Series([similarity[counter][list(range(before_intervention, counter)) + list(range(counter + 1, before_intervention + after_intervention))].mean() for counter in range(before_intervention, before_intervention + after_intervention)]).mean()
        w_similarity_scores.loc[len(w_similarity_scores) + 1, ['username', 'before_after', 'image_similarity']] = [consumer, 'after', after_similarity]

        #w_similarity_scores.to_sql("image_similarity_within_tags", engine, index=None, if_exists='append')    

# load consumers data and extract year and month from dates            
consumers_before_after_query = "SELECT c.username, media1, \
CASE WHEN cc.country = 'canada' AND c.date > '2019-04-30' THEN 'after' \
WHEN c.date > '2019-07-17' THEN 'after' \
ELSE 'before' END as before_after \
FROM consumers_posts c \
INNER JOIN consumers_psm cp ON c.username = cp.username \
INNER JOIN consumers_country cc ON c.username = cc.username \
WHERE c.date > '2018-04-30' AND c.date < '2020-04-30'"

cursor.execute(consumers_before_after_query)
consumers_before_after = pd.DataFrame(cursor.fetchall())
consumers_before_after.columns = [desc[0] for desc in cursor.description]

# compute within-subject similarity 
within_subject_similarity(consumers_before_after, consumers_before_after['username'].unique())

*Between-subjects similarity*  
To assess between-subjects similarity (B) we distinguish between cohorts of users in the treatment and control group. We choose for these comparisons for two reasons. First, Instagram users may especially stay on top of the trends in their local market and therefore their postings might have already been more like other treatment units prior to the intervention. Second, by defining cohorts we establish more homogeneous clusters of users. Within these two cohorts, we determine the cosine similarity of each user pair ($u_i, u_j$) in k = {before, after} (i.e., white squares in figure below). That is, how similar pictures from user i are to pictures from another user j on average, where $u_i$  and $u_j$  belong to the same cohort.

$B_{ijk} = \frac{1}{n_i n_j} \sum_{r=1}^{n} \sum_{c=1}^{n} sim(r_{ijk},c_{ijk})$

The row and column names represent pictures before the intervention for user i ($u_i$) and user j ($u_j$) in the same cohort. Values in the matrix denote the cosine similarity for each picture pair (note: values are left out for simplicity). For the purpose of this analysis we restrict ourselves to the top right or the bottom left white square. Within this area we compute row means to determine how similar a given picture from $u_i$ [$u_j$] is to all pictures from $u_j$ [$u_i$] on average. Thereafter, we sum up the row means and divide by the number of rows ($n_{ik}$ [$n_{jk}$]) to derive the before between-subjects similarity. In a similar fashion, the after between-subjects similarity can be determined. Note that only one of both white squares should be used to avoid duplicates.

<img src="./images/between_subjects_similarity.png" width="500px" align="left"/>

In [9]:
def create_image_output(consumer, consumers, before_after='before'):
    '''construct cosine similarity matrix for either all images before or after the intervention'''
    image_input = load_data("image_tags WHERE username = '" + consumer + "'")
    consumers_df = pd.merge(image_input, consumers, left_on='uri', right_on='media1')[['uri', 'category', 'score', 'before_after', 'treatment_control']]
    tags_matrix = consumers_df.pivot_table(index=["before_after", "uri"], columns="category")
    tags_matrix = tags_matrix.fillna(0)
    if before_after == 'before':
        tags_matrix = tags_matrix.loc[('before')]
    else: 
        tags_matrix = tags_matrix.loc[('after')]    
    return tags_matrix


def between_subjects_similarity(consumers):
    '''compute the between-subjects similarity from image tags before and after the intervention'''
    for consumer1 in consumers.username.unique():
        for consumer2 in consumers.username.unique(): 
            b_similarity_scores = pd.DataFrame(columns = ['username1' , 'username2', 'username1_2', 'before_after', 'similarity']) 
           
            try: 
                if consumer1 != consumer2: # do not compare image data of the same user
                    for before_after in ['before', 'after']: # run this procedure for images before and after the intervention separately
                        consumer1_df = create_image_output(consumer1, consumers, before_after)
                        consumer2_df = create_image_output(consumer2, consumers, before_after)
                        consumers_1_2 = pd.concat([consumer1_df, consumer2_df])
                        consumers_1_2 = consumers_1_2.fillna(0)
                        similarity = cosine_similarity(consumers_1_2)

                        # for each image of consumer 1 take the mean cosine similarity with all image of consumer 2
                        comparisons = [similarity[counter][len(consumer1_df):].mean() for counter in range(len(consumer1_df))]

                        # aggregate the results across all images of consumer 1 (so take the mean of all mean cosine similarities) 
                        before_after_similarity = pd.Series(comparisons).mean() 
                        b_similarity_scores = b_similarity_scores.append({
                                                                "username1": consumer1,
                                                                "username2": consumer2,
                                                                "username1_2": "_".join(sorted([consumer1, consumer2])),
                                                                "before_after": before_after, 
                                                                "similarity": before_after_similarity
                                                                }, ignore_index=True)

                #b_similarity_scores.to_sql("image_similarity_between_tags", engine, index=None, if_exists='append')

            except: 
                pass        

# load consumers data and extract year and month from dates            
connection = engine.connect()
consumers_before_after_treatment_control_query = "SELECT c.username, media1, CASE WHEN cc.country = 'canada' AND c.date > '2019-04-30' THEN 'after' WHEN c.date > '2019-07-17' THEN 'after' ELSE 'before' END as before_after, CASE WHEN cc.country IN ('australia', 'canada', 'italy') THEN 'treatment' ELSE 'control' END as treatment_control FROM consumers_posts c INNER JOIN consumers_psm cp ON c.username = cp.username INNER JOIN consumers_country cc ON c.username = cc.username WHERE c.date > '2018-04-30' AND c.date < '2020-04-30'"
consumers_before_after_treatment_control = connection.execute(consumers_before_after_treatment_control_query).fetchall()
consumers = pd.DataFrame(consumers_before_after_treatment_control).rename({0: 'username', 1: 'media1', 2: 'before_after', 3: 'treatment_control'}, axis=1)

# compute between-subjects similarity (takes a while to run!)
# between_subjects_similarity(consumers)

<a id='outlier-screening'></a>
### F. Outlier Screening
Even though we use a stringent [Instagram consumers selection](#instagram-consumers-selection) procedure, it may sporadically occur that a user systematically differs in terms of the number of followers, followings, and the average number of likes of their image posts. To overcome this issue we use a multivariate outlier screening approach and remove these users from our sample before propensity score matching.

In [10]:
%%R
# split query into subqueries
query_1 = "SELECT x.username, CAST(followers_count AS NUMERIC), CAST(following_count AS NUMERIC), average_likes FROM" 
query_2 = "x INNER JOIN (SELECT username, AVG(total_likes) as average_likes FROM"
query_3 = "l GROUP BY username) l ON x.username = l.username"

query_user_data = function(user_profile, user){
    # paste queries and determine follower count, following count, and average number of likes by user
    query = paste(query_1, user_profile, query_2, user, query_3)
    return(dbGetQuery(con, query))
}

outlier_screening = function(df){
    # determine if the mahalanobis distance exceeds the threshold value
    mahal = mahalanobis(df[,-1], colMeans(df[,-1]), cov(df[,-1]), tol=1e-20)
    cutoff = qchisq(1-0.001, ncol(df[,-1]))
    outliers = subset(df, mahal > cutoff)    
    no_outliers = subset(df, mahal < cutoff)
    return(c(outliers, no_outliers))
}


remove_record = function(table, usernames){
    # remove all posts of users labeled as outliers
    for(username in usernames){
        del_query = paste("DELETE FROM ", table, " WHERE username='", username, "'", sep="")
        dbGetQuery(con, del_query)
    }
}

# collect user stats and then screen for outliers
consumers_stats = query_user_data("consumers_profile", "consumers_posts")
consumers_screening = outlier_screening(consumers_stats)    

# remove outliers from analysis (don't run this cell twice to avoid removing outliers after already excluding outliers)
# remove_record("consumers", consumers_screening[1])

<a id='propensity-score-matching'></a>
### G. Propensity Score Matching
To reduce bias of distribution overlap and different density weighting, we rebalance our data through matching non-treated users to treated ones on similar covariate values. First, we estimate a probit model of receiving treatment on the number of followers, the number of followings, the adoption speed of Instagram users, and the percentage of image posts relative to all types of media posts. Second, we compute the Mahalanobis distance for each treated and control user pair and select unique matches sequentially, in order of closeness of their Mahalanobis distances. We match without replacement such that control units are only allowed to be used as a match once. Each treatment unit is matched with a single control unit as a higher number of matches deteriorates matching quality significantly. Third, we conduct an imbalance check before and after matching of which the results are reported in the paper. 

In [11]:
%%R
# prepare data for propensity score matching (treatment/control country), follower/following count, image share, days since adoption
consumers_query = 
"
SELECT w.username, treatment_control, CAST(followers_count AS NUMERIC), CAST(following_count AS NUMERIC), image, days_since_adoption 
FROM consumers_profile w 
INNER JOIN 
    (SELECT username, CASE     
        WHEN country in ('australia', 'brazil', 'canada', 'italy') THEN 1   
        ELSE 0 END as treatment_control 
    FROM consumers_country cc WHERE country != 'brazil') x ON w.username = x.username

INNER JOIN (SELECT w.username, AVG(CAST(image_count AS DECIMAL) / CAST(posts_count AS DECIMAL)) as image 
            FROM consumers_profile w 
            INNER JOIN (SELECT username, SUM(CASE WHEN content_type = 'GraphImage' THEN 1 END) as image_count 
            FROM consumers_posts GROUP BY username) l ON l.username = w.username GROUP BY w.username) y ON y.username = w.username 

INNER JOIN (SELECT username, DATE_PART('day', '2020-06-01'::timestamp - MIN(timestamp)) as days_since_adoption 
            FROM consumers_posts GROUP BY username) z ON z.username = y.username
"

PSM = function(df){  
    # propensity score matching
    Tr = cbind(as.vector(df$treatment_control))
    X = as.matrix(df[,c('followers_count', 'following_count', 'days_since_adoption', 'image')])
    
    # replace NAs in the image column with zero
    X[is.na(X)] = 0 
    
    glm1 = glm(Tr ~ X, family=binomial)
    
    rr1 = Match(Tr=Tr, X=glm1$fitted, replace = FALSE, Weight=1, M=1)
    summary(rr1)
  
    # check balancing properties (results may deviate between bootstrap iterations)
    MatchBalance(Tr ~ X, match.out = rr1, nboots=10000)
  
    # store indices of matched users
    treatment = data.frame(df[rr1$index.treated,'username'], 'treatment')
    colnames(treatment) = c("username", "type")
    control = data.frame(df[rr1$index.control,'username'], 'control')
    colnames(control) = c("username", "type")
    return(rbind(treatment, control))
}

consumers_PSM_input = dbGetQuery(con, consumers_query)

# lines below are commented to ensure consistency with paper (PSM results may slightly deviate for each run)
# consumers_PSM = PSM(consumers_PSM_input)
# dbWriteTable(con, "consumers_psm", consumers_PSM, overwrite = TRUE, row.names = FALSE) 

<img src="./images/instagram_header.png" align="left"/>

*Klaasse Bos, R.J. (2020). Web Appendix: Goodbye Likes, Hello Mental Health: How Hiding Like Counts Affects User Behavior & Self-Esteem.*